# Pipeline de IA com a Base de Dados do TitanicLista 11 - Inteligência Artificial

## 1. Importação de Bibliotecas e Carregamento da Base

In [ ]:
import pandas as pdimport numpy as npimport seaborn as snsimport matplotlib.pyplot as pltfrom sklearn.model_selection import train_test_splitfrom sklearn.preprocessing import StandardScaler, LabelEncoderfrom sklearn.ensemble import RandomForestClassifierfrom sklearn.tree import DecisionTreeClassifierfrom sklearn.metrics import classification_report, confusion_matrixfrom sklearn.decomposition import PCAfrom sklearn.cluster import KMeansfrom mlxtend.frequent_patterns import apriori, association_rules# Carregando os dadosdf = pd.read_csv('https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv')df.head()

## 2. Análise Exploratória e Pré-Processamento

In [ ]:
# Verificando informações geraisdf.info()# Estatísticas descritivasdf.describe()# Valores nulosdf.isnull().sum()

In [ ]:
# Preenchendo valores nulosdf['Age'].fillna(df['Age'].median(), inplace=True)df['Embarked'].fillna(df['Embarked'].mode()[0], inplace=True)df.drop(columns=['Cabin'], inplace=True)  # Muitas informações faltantes# Extraindo título dos nomesdf['Title'] = df['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)df['Title'] = df['Title'].replace(['Mlle', 'Ms'], 'Miss')df['Title'] = df['Title'].replace(['Mme', 'Lady', 'Countess', 'Dona'], 'Mrs')df['Title'] = df['Title'].replace(['Capt', 'Col', 'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer'], 'Rare')# Criando variável "FamilySize"df['FamilySize'] = df['SibSp'] + df['Parch'] + 1# Codificação de variáveis categóricasdf = pd.get_dummies(df, columns=['Sex', 'Embarked', 'Title'], drop_first=True)# Removendo colunas irrelevantesdf.drop(columns=['PassengerId', 'Name', 'Ticket'], inplace=True)# Separando variáveisX = df.drop('Survived', axis=1)y = df['Survived']# Normalizando variáveis numéricasscaler = StandardScaler()X_scaled = scaler.fit_transform(X)

## 3. Modelagem Supervisionada (Classificação)

In [ ]:
# Dividindo os dadosX_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)# Random Forestrf = RandomForestClassifier()rf.fit(X_train, y_train)y_pred_rf = rf.predict(X_test)# Decision Treedt = DecisionTreeClassifier()dt.fit(X_train, y_train)y_pred_dt = dt.predict(X_test)# Avaliaçãoprint("Random Forest:", classification_report(y_test, y_pred_rf))print("Decision Tree:", classification_report(y_test, y_pred_dt))

## 4. Modelagem Não Supervisionada (Clusterização)

In [ ]:
# Aplicando PCA para visualizaçãopca = PCA(n_components=2)X_pca = pca.fit_transform(X_scaled)# KMeanskmeans = KMeans(n_clusters=2, random_state=42)clusters = kmeans.fit_predict(X_scaled)# Visualizandoplt.figure(figsize=(10,6))sns.scatterplot(x=X_pca[:,0], y=X_pca[:,1], hue=clusters, palette='viridis')plt.title("Clusters de Passageiros (K-Means + PCA)")plt.show()

## 5. Regras de Associação

In [ ]:
# Selecionando colunas categóricas binarizadasassoc_df = df.copy()assoc_df['Survived'] = assoc_df['Survived'].astype(bool)# Binarizando as variáveis para Aprioriassoc_df_bin = assoc_df.copy()assoc_df_bin = assoc_df_bin.astype(int).astype(bool)# Aplicando Apriorifrequent_itemsets = apriori(assoc_df_bin, min_support=0.1, use_colnames=True)rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.0)# Exibindo 3 regras com maior liftrules.sort_values(by="lift", ascending=False).head(3)

## 6. Conclusão

Com base na análise, os principais insights foram:- O modelo Random Forest teve melhor desempenho que a árvore de decisão.- A variável 'Title' e o tamanho da família foram relevantes na predição de sobrevivência.- A clusterização revelou agrupamentos distintos com base no perfil dos passageiros.- As regras de associação indicam padrões importantes, como alta sobrevivência de mulheres de 1ª classe.O pipeline apresentado demonstra como aplicar técnicas supervisionadas, não supervisionadas e de associação em dados reais.